# Retreiving and Preparing Stock's Cash Flow Statement
This section of the code will concentrate on extracting stock's cash flow statement from IEX Cloud API. At the end, this section of the code will provide a complete and formatted cash flow dataset.  

## Importing neccessary modules

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

## Setting up base URL

Fetching the fundamentals and finacial data about the stock from ther IEX Cloud endpoints.

In [19]:
from api_secret import IEX_CLOUD_API_TOKEN

symbol = 'FTNT'# input("Enther the ticker here: ")

# API endpoints to collect stock data
cash_flow_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/cash-flow?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
financials_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/financials?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
fundamentals_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/fundamentals?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'


cash_flow_endpoint_data = requests.get(cash_flow_endpoint).json()
financials_endpoint_data = requests.get(financials_endpoint).json()
fundamentals_endpoint_data = requests.get(fundamentals_endpoint).json()


## Extracting the data
This part will concentrate on extracting the data that will form the final cash flow statement for the stock.

First step is to find the quarters of the reported stock data.

In [23]:
reportDate_list = []
quarter_list = []

# Converting string date into the date object
for stock_data in cash_flow_endpoint_data['cashflow']:
    converted_date = datetime.strptime(stock_data['reportDate'], '%Y-%m-%d').date()
    reportDate_list.append(converted_date)


# function to determine to which quarter the reported stock data
# is referred to
def quarter_finder(dates_list):
    global quarter_list

    for d in dates_list:
        q1 = date(d.year, 3, 31)
        q2 = date(d.year, 6, 30)
        q3 = date(d.year, 9, 30)
        q4 = date(d.year, 12, 31)

        if d < q1:
            q = date(d.year - relativedelta(year=1).year, 12, 31)
        if d > q1 and d < q2:
            q = date(d.year, 3, 30)
        if d > q2 and d < q3:
            q = date(d.year, 6, 30)
        if d > q3 and d < q4:
            q = date(d.year, 9, 30)
        
        quarter_list.append(q)

quarter_finder(reportDate_list)

The above created Datetime-like array that will serve as an input to Pandas' DatetimeIndex objet (columns of the future dataset)

In [24]:
columns = pd.DatetimeIndex(data=quarter_list)

The next step is the creation of the actual DataFrame.

In [33]:
index = [
    'Net Income',
    'Net Income (Growth)',
    'Depreciation',
    'Stock based compensation',
    'Accounts receivable',
    'Accounts receivable (Growth)',
    'Changes in Inventories',
    'Accounts Payable',
    'Accounts Payable (Growth)',
    'Net Operating Cash Flow',
    'Net Operating Cash Flow (Growth)',
    'Capital Expenditures',
    'Capital Expenditures (Growth)',
    'Net Assets from Acquisitions',
    'Purchase/Sale of investments',
    'Net Investing Cash Flow',
    'Common Stock Repurchased',
    'Dividends Payed',
    'Other Financing Activities',
    'Net Financing Cash Flow',
    'Net Financing Cash Flow (Growth)'
]

cash_flow_statement = pd.DataFrame(columns=columns, index=index, data=None)

### Filling the dataframe with data returned by API endpoints.

The first API endpoint to get the stock data from is the 'financials' endpoint.

In [29]:
financials_dict = {
    'Accounts Payable': []
}


# Filling the financials_dict dictionary with data collected
# from the API endpoint
for quarter_data in financials_endpoint_data['financials']:
    financials_dict['Accounts Payable'].append(quarter_data['totalRevenue'])


# Putting the data into the DataFrame
for stock_attribute in financials_dict:
    for row in cash_flow_statement.index:
        if row == stock_attribute:
            cash_flow_statement.loc[row] = financials_dict[stock_attribute]


The second is the income API endpoint.

In [32]:
cash_flow_dict = {
    'Net Income': [],
    'Depreciation': [],
    'Accounts receivable': [],
    'Changes in Inventories': [],
    'Net Operating Cash Flow': [],
    'Capital Expenditures': [],
    'Purchase/Sale of investments': [],
    'Net Investing Cash Flow': [],
    'Dividends Payed': [],
    'Other Financing Activities': [],
    'Net Financing Cash Flow': []
}

for quarter_data in cash_flow_endpoint_data['cashflow']:
    cash_flow_dict['Net Income'].append(quarter_data['netIncome'])
    cash_flow_dict['Depreciation'].append(quarter_data['depreciation'])
    cash_flow_dict['Accounts receivable'].append(quarter_data['changesInReceivables'])
    cash_flow_dict['Changes in Inventories'].append(quarter_data['changesInInventories'])
    cash_flow_dict['Net Operating Cash Flow'].append(quarter_data['cashFlow'])
    cash_flow_dict['Capital Expenditures'].append(quarter_data['capitalExpenditures'])
    cash_flow_dict['Purchase/Sale of investments'].append(quarter_data['investments'])
    cash_flow_dict['Net Investing Cash Flow'].append(quarter_data['totalInvestingCashFlows'])
    cash_flow_dict['Dividends Payed'].append(quarter_data['dividendsPaid'])
    cash_flow_dict['Other Financing Activities'].append(quarter_data['otherFinancingCashFlows'])
    cash_flow_dict['Net Financing Cash Flow'].append(quarter_data['cashFlowFinancing'])
    

for stock_attribute in cash_flow_dict:
    for row in cash_flow_statement.index:
        if row == stock_attribute:
            cash_flow_statement.loc[row] = cash_flow_dict[stock_attribute]

The final endpoint to use the data from in the dataset is the fundamentals endpoint.

In [35]:
fundamentals_dict = {
    'Stock based compensation': [],
    'Net Assets from Acquisitions': [],
    'Common Stock Repurchased': []
}

for quarter_data in fundamentals_endpoint_data['fundamentals']:
    fundamentals_dict['Stock based compensation'].append(quarter_data['expensesStockCompensation'])
    fundamentals_dict['Net Assets from Acquisitions'].append(quarter_data['capexAcquisition'])
    fundamentals_dict['Common Stock Repurchased'].append(quarter_data['cashFlowShareRepurchase'])

for stock_attribute in fundamentals_dict:
    for row in cash_flow_statement.index:
        if row == stock_attribute:
            cash_flow_statement.loc[row] = fundamentals_dict[stock_attribute]

## Calculating the Growth

Below section deals with the 'Growth' columns to give an investor an advantage of observing company's success metric.

In [36]:
# Calculating the growth rate based on the stock metrics' 
for df_index, row_name in enumerate(cash_flow_statement.index):
    if 'Growth' in row_name:
        for row_index, value in enumerate(cash_flow_statement.iloc[df_index - 1]):
            try:
                growth_rate = (value / cash_flow_statement.iloc[df_index - 1, row_index + 1]) * 100
                cash_flow_statement.iloc[df_index, row_index] = (growth_rate - 100) / 100
            except IndexError:
                break

cash_flow_statement

,2022-09-30,2022-06-30,2022-03-30,2021-12-31,2021-09-30,2021-06-30,2021-03-30,2020-12-31,2020-09-30,2020-06-30,2020-03-30,2019-12-31
Net Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net Income (Growth),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Stock based compensation,0,0,0,0,0,0,0,0,0,0,0,0
Accounts receivable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accounts receivable (Growth),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Changes in Inventories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accounts Payable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accounts Payable (Growth),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net Operating Cash Flow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
